## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-26-23-49-52 +0000,nypost,Nike x SKIMS launch fizzles in NYC: Kim K coll...,https://nypost.com/2025/09/26/lifestyle/nike-x...
1,2025-09-26-23-49-31 +0000,nypost,Trump’s TikTok deal still worries GOP China h...,https://nypost.com/2025/09/26/business/trumps-...
2,2025-09-26-23-46-25 +0000,wapo,Jimmy Kimmel is coming back to Nexstar and Sin...,https://www.washingtonpost.com/business/2025/0...
3,2025-09-26-23-44-33 +0000,bbc,Former FBI director James Comey indicted on tw...,https://www.bbc.com/news/articles/cy50ggv35zpo...
4,2025-09-26-23-44-04 +0000,nypost,Trump demands Microsoft fire ex-Biden DOJ No. ...,https://nypost.com/2025/09/26/us-news/trump-de...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,69
33,comey,25
142,new,23
176,will,20
133,tariffs,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
127,2025-09-26-18-49-37 +0000,nypost,Trump predicts ‘there will be others’ after Ja...,https://nypost.com/2025/09/26/us-news/trump-pr...,149
332,2025-09-25-23-56-17 +0000,nypost,Trump celebrates James Comey indictment in sca...,https://nypost.com/2025/09/25/us-news/trump-ce...,147
168,2025-09-26-16-22-19 +0000,bbc,Watch: Trump says 'there will be others' after...,https://www.bbc.com/news/videos/ceq2zzp1dx2o?a...,133
88,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...,132
34,2025-09-26-22-28-20 +0000,nypost,Trump backs plan to make ex-UK PM Tony Blair ‘...,https://nypost.com/2025/09/26/world-news/trump...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
127,149,2025-09-26-18-49-37 +0000,nypost,Trump predicts ‘there will be others’ after Ja...,https://nypost.com/2025/09/26/us-news/trump-pr...
167,78,2025-09-26-16-22-23 +0000,nypost,"Netanyahu shrugs off UN walkout, says Israel m...",https://nypost.com/2025/09/26/us-news/netanyah...
88,63,2025-09-26-20-38-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...
32,52,2025-09-26-22-32-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...
84,52,2025-09-26-20-43-00 +0000,wsj,Even as Senate Minority Leader Chuck Schumer s...,https://www.wsj.com/politics/policy/government...
34,48,2025-09-26-22-28-20 +0000,nypost,Trump backs plan to make ex-UK PM Tony Blair ‘...,https://nypost.com/2025/09/26/world-news/trump...
63,40,2025-09-26-21-25-31 +0000,nyt,Sinclair and Nexstar End Jimmy Kimmel Boycott ...,https://www.nytimes.com/2025/09/26/business/me...
319,39,2025-09-26-01-00-00 +0000,wsj,The Supreme Court is poised to decide whether ...,https://www.wsj.com/economy/central-banking/fe...
37,37,2025-09-26-22-25-50 +0000,nypost,ICE agent who shoved immigrant in viral NYC co...,https://nypost.com/2025/09/26/us-news/ice-agen...
68,37,2025-09-26-21-20-00 +0000,wsj,Defense Secretary Pete Hegseth is convening hu...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
